# Farmer problem

Reference: Birge and Louveaux, "Introduction to Stochastic Programming", Chapter 1

In [ ]:
using JuMP

In [ ]:
# Open-source solvers
using GLPK
using HiGHS

Commercial solvers as `CPLEX` or `Gurobi` can also be used.

Among open-source solvers, we recommend HiGHS ((https://highs.info/), as suggested by the presentation done by Julian Hall at JuliaCon 2022: https://www.youtube.com/watch?v=ChKdTqUzXdY

# Average Scenario

We construct the model for the average scenario, using first the open-source HiGHS solver .

In [ ]:
m = Model(HiGHS.Optimizer)
#m = Model(GLPK.Optimizer)

crops = ["Wheat", "Corn", "Sugar beets"]
ncrops = length(crops)

@variable(m, x[1:ncrops] >= 0)
@variable(m, y[1:2] >= 0)
@variable(m, w[1:4] >= 0)

costSeeding = [150, 230, 260]
costs = [238, 210]
prices = [170, 150, 36, 10]

We can check that `m` is a JuMP model by checking its type.

In [ ]:
typeof(m)

In [ ]:
typeof(x)

We now add the constraints.

In [ ]:
@constraint(m, surface, sum(x[i] for i=1:ncrops) <= 500)

In [ ]:
@constraint(m, wheatNeeds, 2.5x[1]+y[1]-w[1] >= 200)

In [ ]:
@constraint(m, cornNeeds, 3x[2]+y[2]-w[2] >= 240)

In [ ]:
@constraint(m, beetsProd, w[3]+w[4] <= 20x[3])

In [ ]:
@constraint(m, beetsQuota, w[3] <= 6000)

Let's define the objective function. 

In [ ]:
@objective(m, Min, sum(costSeeding[i]*x[i] for i=1:ncrops) + sum(costs[i]*y[i] for i = 1:length(costs))
    - (sum(prices[i]*w[i] for i=1:length(prices))))

The complete model can be displayed with the command `print`.

In [ ]:
print(m)

More information can be obtained with the method `show`:

In [ ]:
show(m)

We now solve it.

In [ ]:
optimize!(m)

In [ ]:
print("Solution: $(value.(x))")

In [ ]:
? value

In [ ]:
methods(value)

In [ ]:
value(surface)

We will summarize the solution approach using a Julia function.

In [ ]:
function farmer(factor::Float64 = 1.0, m::Model = Model(HiGHS.Optimizer))
    crops = ["Wheat", "Corn", "Suger beets"]
    ncrops = length(crops)

    @variable(m, x[1:ncrops] >= 0)
    @variable(m, y[1:2] >= 0)
    @variable(m, w[1:4] >= 0)

    costSeeding = [150, 230, 260]
    costs = [238, 210]
    prices = [170, 150, 36, 10]
    returns = factor*[2.5, 3, 20]

    @constraint(m, surface, sum(x[i] for i=1:3) <= 500)

    @constraint(m, wheatNeeds, returns[1]x[1]+y[1]-w[1] >= 200)
    @constraint(m, cornNeeds, returns[2]x[2]+y[2]-w[2] >= 240)
    @constraint(m, beetsProd, w[3]+w[4] <= returns[3]x[3])
    @constraint(m, beetsQuota, w[3] <= 6000)

    @objective(m,Min,sum(costSeeding[i]*x[i] for i=1:3) + sum(costs[i]*y[i] for i = 1:length(costs))
    - (sum(prices[i]*w[i] for i=1:length(prices))))
    
    status = optimize!(m)

    # println(value.(x))

    return m, status
end

In [ ]:
methods(farmer)

In [ ]:
maverage, status = farmer()

In [ ]:
maverage

In [ ]:
print(maverage)

In [ ]:
objective_value(maverage)

We can recover the decision variables with the method `all_variables`:

In [ ]:
variables = all_variables(maverage)

We can thus get the first-stage decisions.

In [ ]:
value.(variables[1:3])

## Good scenario

In [ ]:
mgood, status = farmer(1.2)

In [ ]:
mgood

In [ ]:
objective_value(mgood)

## Bad scenario

In [ ]:
mbad, status = farmer(0.8)
#mbad, status = farmer(0.8, Model(solver = GurobiSolver()))

In [ ]:
objective_value(mbad)

## Stochastic program - extensive form

We have to combine the three scenarios. We will assume that each one has a probability equal to 1/3.

In [ ]:
n = 3
p = [1/n for i = 1:n]

We will start with a naive implementation.

In [ ]:
function farmerStoch()
    m = Model(HiGHS.Optimizer)

    crops = ["Wheat", "Corn", "Suger beets"]
    ncrops = length(crops)

    @variable(m,x[1:ncrops] >= 0)

    @variable(m,ya[1:2] >= 0)
    @variable(m,wa[1:4] >= 0)

    @variable(m,yg[1:2] >= 0)
    @variable(m,wg[1:4] >= 0)

    @variable(m,yb[1:2] >= 0)
    @variable(m,wb[1:4] >= 0)

    costSeeding = [150, 230, 260]
    costs = [238, 210]
    prices = [170, 150, 36, 10]
    returnsA = [2.5, 3, 20]
    returnsG = 1.2*returnsA
    returnsB = 0.8*returnsA

    @constraint(m, surface, sum(x[i] for i=1:3) <= 500)

    @constraint(m, wheatNA, returnsA[1]x[1]+ya[1]-wa[1] >= 200)
    @constraint(m, cornNA, returnsA[2]x[2]+ya[2]-wa[2] >= 240)
    @constraint(m, beetsPA, wa[3]+wa[4] <= returnsA[3]x[3])
    @constraint(m, beetsQA, wa[3] <= 6000)

    @constraint(m, wheatNG, returnsG[1]x[1]+yg[1]-wg[1] >= 200)
    @constraint(m, cornNG, returnsG[2]x[2]+yg[2]-wg[2] >= 240)
    @constraint(m, beetsPG, wg[3]+wg[4] <= returnsG[3]x[3])
    @constraint(m, beetsQG, wg[3] <= 6000)

    @constraint(m, wheatNB, returnsB[1]x[1]+yb[1]-wb[1] >= 200)
    @constraint(m, cornNB, returnsB[2]x[2]+yb[2]-wb[2] >= 240)
    @constraint(m, beetsPB, wb[3]+wb[4] <= returnsB[3]x[3])
    @constraint(m, beetsQB, wb[3] <= 6000)

    @objective(m, Min, sum(costSeeding[i]*x[i] for i=1:3)
        + 1/3*sum(costs[i]*ya[i] for i = 1:length(costs))
        - 1/3*sum(prices[i]*wa[i] for i=1:length(prices))
        + 1/3*sum(costs[i]*yg[i] for i = 1:length(costs))
        - 1/3*sum(prices[i]*wg[i] for i=1:length(prices))
        + 1/3*sum(costs[i]*yb[i] for i = 1:length(costs))
        - 1/3*sum(prices[i]*wb[i] for i=1:length(prices)))
    
    status = optimize!(m)

    return m, status
end

In [ ]:
mstoch, status = farmerStoch()

In [ ]:
print(mstoch)

In [ ]:
valStoch = objective_value(mstoch)

## Expected value of the perfect information

In [ ]:
values = [objective_value(maverage), objective_value(mgood), objective_value(mbad)]

Let's compute the expected revenue under perfect information, simply obtained as the dot product between the probabilities vector and the optimal values vector.

In [ ]:
valPerfect = p'*values

In [ ]:
using LinearAlgebra

dot(p, values)

In [ ]:
methods(dot)

Expected value of perfect information:

In [ ]:
valPerfect - valStoch

## Value of the stochastic solution

We have to fix the first stage decision. We define the second stage problem.

In [ ]:
function secondStage(x, factor::Float64 = 1.0)
    m = Model(HiGHS.Optimizer)
    
    @variable(m,y[1:2] >= 0)   # purchases
    @variable(m,w[1:4] >= 0)   # sales

    costs = [238, 210]
    prices = [170, 150, 36, 10]
    returns = (factor*[2.5, 3, 20]).*x
    z = [200 - returns[1], 240 - returns[2], returns[3]]
    
    @constraint(m, wheatNeeds, y[1]-w[1] >= z[1])
    @constraint(m, cornNeeds, y[2]-w[2] >= z[2])
    @constraint(m, beetsProd, w[3]+w[4] <= z[3])
    @constraint(m, beetsQuota, w[3] <= 6000)

    @objective(m,Min, sum(costs[i]*y[i] for i = 1:length(costs)) - (sum(prices[i]*w[i] for i=1:length(prices))))
    
    status = optimize!(m)
    
    return m, status
end

In [ ]:
x = [120, 80, 300]

In [ ]:
msecond, status = secondStage(x)

In [ ]:
print(msecond)

In [ ]:
vaverage = objective_value(msecond)

In [ ]:
msecond, status = secondStage(x, 1.2)

In [ ]:
vgood = objective_value(msecond)

In [ ]:
msecond, status = secondStage(x, 0.8)

In [ ]:
vbad = objective_value(msecond)

Expected revenue

In [ ]:
er = costSeeding'*x+p'*[vaverage, vgood, vbad]

Value of the stochastic solution

In [ ]:
vss = valStoch - er